## Designing Smart and Healthy Systems
### HW 1: Exploratory Data Analysis and Data Preparation - Task 4, April 2020

#### Group 4: Batshur (Shuree) Gootiz, Sanjana Kotecha

### EDA of Lab Findings Dataset

In [1]:
# Imports required to load the datasets from S3 Bucket
import os
import boto3
from boto3 import client

# Additional imports required for the homework tasks
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import preprocessing

In [2]:
#PART 4 STARTS here: 
s3_bucket = os.environ.get('DATASET_BUCKET', 'cmu-dshs-diabetes-data')
s3_key3 = ('Balance - Tablas (Parte 3 ).xlsx')
df_key3 = pd.ExcelFile('s3://{}/{}'.format(s3_bucket, s3_key3)) #, encoding = 'ISO-8859-1')

df_key3.sheet_names


['visitas', 'valoracionpodologicaseguimiento', 'valoracionesexpress']

In [3]:
visitas = df_key3.parse('visitas')
visitas.head()
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75725 entries, 0 to 75724
Data columns (total 7 columns):
IdVisita         75725 non-null int64
IdConsultorio    75725 non-null int64
IdPaciente       75725 non-null int64
Fecha            75725 non-null object
Hora             75725 non-null object
Area             75725 non-null object
Usuario          75725 non-null object
dtypes: int64(3), object(4)
memory usage: 4.0+ MB


In [4]:
valor = df_key3.parse('valoracionpodologicaseguimiento')
valor.head()


,IdValoracionPodologicaSeguimiento,IdValoracionPodologica,IdPaciente,Fecha,Hora,Nota,Usuario,Neuropatia,TipoNeuropatia
0,3,2,771,2013-02-08,09:50:00,PACIENTE FEMENINA CON BUENA SENSIBILIDAD PROTE...,39564f3f8482346e4764d24bb0a60260,0.0,PERIFERICA
1,4,4,771,2013-02-08,09:50:00,"PACIENTE FEMENINO, CON SENSIBILIDAD PROFUNDA Y...",39564f3f8482346e4764d24bb0a60260,0.0,PERIFERICA
2,5,5,770,2013-02-07,09:10:00,PACIENTE FEMENINA CON BUENA SENSIBILIDAD PROTE...,39564f3f8482346e4764d24bb0a60260,0.0,PERIFERICA
3,6,6,773,2013-02-11,10:00:00,PACIENTE MASCULINO CON DATOS DE NEUROPATÍA EN ...,39564f3f8482346e4764d24bb0a60260,0.0,PERIFERICA
4,7,2,774,2013-02-12,16:05:00,"RIESGO DE PIE DIABÉTICO: BAJOPULSOS PRESENTES,...",39564f3f8482346e4764d24bb0a60260,0.0,PERIFERICA


In [5]:

#This seems to be related to lab findings
valor_express = df_key3.parse('valoracionesexpress')
valor_express.info()
valor_express.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42944 entries, 0 to 42943
Data columns (total 90 columns):
IdProspecto                   42944 non-null int64
Fecha                         42944 non-null object
Hora                          42944 non-null object
MotivoValoracion              42943 non-null object
CualMotivoExpress             8834 non-null object
PresionArterial               13687 non-null object
GlicemiaCapilar               42943 non-null float64
Peso                          42943 non-null float64
Talla                         42943 non-null float64
PlanAlimentacion              42944 non-null int64
Examen de Ojos                42944 non-null int64
ValoracionPies                42944 non-null int64
ExamenesLaboratorio           42944 non-null int64
HbA1c                         42944 non-null int64
Colesterol                    42944 non-null int64
Trigliceridos                 42944 non-null int64
Microalbuminuria              42944 non-null int64
Otras         

,IdProspecto,Fecha,Hora,MotivoValoracion,CualMotivoExpress,PresionArterial,GlicemiaCapilar,Peso,Talla,PlanAlimentacion,...,GlicemiaCapilar12,Peso12,Talla12,GlicemiaCapilarTipo12,HA_SIS12,HA_DIA12,LevantamientoRosa12,EntregaCompleto12,FirmaRecibido12,Fecha1c12
0,5,2013-03-12 00:00:00,09:44:00,DETECCION,TIPO 2,124/75,371.0,70.9,1.51,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,8,2013-03-12 00:00:00,11:23:00,EXPRESS,TIPO 2,124/74,224.0,52.5,1.49,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,9,2013-03-12 00:00:00,14:32:00,EXPRESS,DM2,156/82,181.0,53.8,1.55,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,10,2013-03-12 00:00:00,18:32:00,EXPRESS,DMT2,145/86,227.0,92.0,1.76,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,11,2013-03-13 00:00:00,17:35:00,EXPRESS,NaN,123/67,108.0,79.4,28.10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [6]:
#Even though there 90 columns, many of them have missing values, some 100% of values missing. 
#So created the summary table for missing values

percent_missing = valor_express.isnull().sum() * 100 / len(valor_express)
missing_value_df = pd.DataFrame({'column_name': valor_express.columns,
                                 'percent_missing': percent_missing})

missing_value_df.sort_values('percent_missing', inplace=True)

#Looking at table in which missing values are less than 40%. From here, I made a selection. 
missing_value_df[missing_value_df.percent_missing < 40]

,column_name,percent_missing
IdProspecto,IdProspecto,0.000000
EOtros,EOtros,0.000000
Usuario,Usuario,0.000000
Otras,Otras,0.000000
Microalbuminuria,Microalbuminuria,0.000000
Colesterol,Colesterol,0.000000
HbA1c,HbA1c,0.000000
ExamenesLaboratorio,ExamenesLaboratorio,0.000000
ValoracionPies,ValoracionPies,0.000000
Examen de Ojos,Examen de Ojos,0.000000


#### Lab Findings Predictors

Used translations to understand the same


Assumption: Blood Sugar Level we are not sure how it is checked as it can be checked in lab + can be checeked using a physical devise

In [15]:
#Create dataframe for lab findings with 10-15 lab measures; based on relevance and less missing values
lab_findings_df =valor_express[['IdProspecto', 'ExamenesLaboratorio', 'HbA1c', 'Colesterol', 'GlicemiaCapilar', 'Trigliceridos', 'Microalbuminuria','EHBA1C','Electrocardiograma', 'QuimicaSanguinea','PerfilLipidos']]
lab_findings_df.head(10)

,IdProspecto,ExamenesLaboratorio,HbA1c,Colesterol,GlicemiaCapilar,Trigliceridos,Microalbuminuria,EHBA1C,Electrocardiograma,QuimicaSanguinea,PerfilLipidos
0,5,1,1,1,371.0,1,1,5.0,0.0,0.0,0.0
1,8,1,1,1,224.0,1,1,NaN,NaN,NaN,NaN
2,9,0,0,0,181.0,0,0,NaN,NaN,NaN,NaN
3,10,0,0,0,227.0,0,0,NaN,NaN,NaN,NaN
4,11,0,0,0,108.0,0,0,NaN,NaN,NaN,NaN
5,12,1,1,1,600.0,1,1,NaN,NaN,NaN,NaN
6,14,0,0,0,186.0,0,0,NaN,NaN,NaN,NaN
7,16,0,0,0,281.0,0,0,NaN,NaN,NaN,NaN
8,18,1,1,1,132.0,1,1,NaN,NaN,NaN,NaN
9,21,0,0,0,132.0,0,0,NaN,NaN,NaN,NaN


Pushing the lab finding predictors to csv

In [ ]:
lab_findings_df.to_csv (r'lab_findings_df.csv', index = False, header=True)

#### Physical Examination Predictors

Used translations to understand the same

The fields we chose below are

1. Glycemia Capillary - Underatsnding that this helps in testing blood sugar but can be done with a handy device
2. Peso - Weight
3. Talla - Height
4. PlanAlimentacion - Food Plan (Yes/No)
5. Eye Examination
6. Feet Rating
7. Systolic Pressure
8. Diastolic Pressure
9. Physical Activity
10. Blood Pressure history
11. Diabetes History

However, there are missing values for teh last 3 columns and hence we need to choose a wise way to handle the same

In [11]:
#Create dataframe for physical exam predictors with 10-15 measures; based on relevance and less missing values
lab_findings_df =valor_express[['IdProspecto','GlicemiaCapilar', 'Peso', 'Talla', 'PlanAlimentacion', 'Examen de Ojos', 
                                'ValoracionPies','HA_SIS', 'HA_DIA', 'ActividadFisica', 'HA_Antecedentes', 'DIAB_Antecedentes']]
lab_findings_df.head(10)

,IdProspecto,GlicemiaCapilar,Peso,Talla,PlanAlimentacion,Examen de Ojos,ValoracionPies,HA_SIS,HA_DIA,ActividadFisica,HA_Antecedentes,DIAB_Antecedentes
0,5,371.0,70.9,1.51,1,1,0,124.0,75.0,0.0,0.0,0.0
1,8,224.0,52.5,1.49,1,1,1,124.0,74.0,NaN,NaN,NaN
2,9,181.0,53.8,1.55,0,0,0,156.0,82.0,NaN,NaN,NaN
3,10,227.0,92.0,1.76,0,0,0,145.0,86.0,NaN,NaN,NaN
4,11,108.0,79.4,28.10,1,0,0,123.0,67.0,NaN,NaN,NaN
5,12,600.0,91.0,1.85,1,1,1,144.0,101.0,NaN,NaN,NaN
6,14,186.0,70.3,1.58,0,0,0,123.0,64.0,NaN,NaN,NaN
7,16,281.0,73.5,1.65,1,0,0,203.0,102.0,NaN,NaN,NaN
8,18,132.0,69.2,1.61,1,1,1,136.0,79.0,NaN,NaN,NaN
9,21,132.0,83.4,1.56,0,0,0,169.0,89.0,NaN,NaN,NaN


In [12]:
lab_findings_df.to_csv (r'physical_examination_df.csv', index = False, header=True)